# Goal: Describe CCG opiate prescribing in England using the GP prescribing data.

In [ ]:
import os, datetime
import pandas as pd
import pickle
import folium
from pandas import Series, DataFrame, Panel
from datetime import datetime, date, time

In [ ]:
# Data sources:
# wget all the presentation prescribing datas here http://www.hscic.gov.uk/gpprescribingdata
# get gp pop sizes and gp-ccg mapping here https://indicators.ic.nhs.uk/download/Clinical%20Commissioning%20Group%20Indicators/Data/GP_registered_patients_2012.csv
# get ccg pop sizes here https://indicators.ic.nhs.uk/download/Clinical%20Commissioning%20Group%20Indicators/Data/CCG_registered_patients_2012.csv
# get map boundary here https://geoportal.statistics.gov.uk/Docs/Boundaries/Clinical_commissioning_groups_(Eng)_Apr_2013_Boundaries_(Full_Extent).zip


In [ ]:
# cuts gp prescribing data for rows containing patterns of interest

def pattern_timeseries(pattern):
    
    pathtodata = '/media/drcjar/pidisk/prescribing_data/bnf/' # set to the directory with bnf CSVs in
    pathtopatterns = '/media/drcjar/pidisk/prescribing_data/patterns/' 
    pathtopickles = '/media/drcjar/pidisk/prescribing_data/pickles/'
    
    os.chdir(pathtodata)
    
    files = !ls T2*
    
    global pattern_df
    
    clean_filenames = []

    for f in files.l:
        clean_filenames.append(f[:7]) # clean up filenames so that grep will work
        
    for i, item in enumerate(clean_filenames):
        date = clean_filenames[i].replace('T', '')
        date = pd.to_datetime(date, format="%Y%m") # make pandas know the date is a date
        name = "bnf_%s_%.10s.csv" % (pattern, date) # create a name for new csv of grep for pattern using date from file name
        print ("writing %s" % name)
        !fgrep $pattern {clean_filenames[i]}* > $pathtopatterns/$name # grep for pattern in csv files and write to file 
    
    pattern_files = !ls $pathtopatterns*$pattern*
    
    cols = ['SHA', 'PCT', 'PRACTICE', 'BNF_CODE', 'BNF_NAME', 'ITEMS', 'NIC', 'ACT_COST', 'QUANTITY', 'DateTime', 'Index']
    
    practices_est = 10000 #estimated number of practices
    
    df_list = [pd.read_csv(file, names=cols) for file in pattern_files] 
            
    pattern_df = pd.concat(df_list)
    
    pattern_df['DateTime'] = pattern_df['DateTime'].astype('|S6') 
    pattern_df['DateTime'] = pd.to_datetime(pattern_df['DateTime'], format="%Y%m")
    
    os.chdir(pathtopickles)

    pattern_df.to_pickle('%s.pkl' % pattern)
                    
    return(pattern_df)


In [ ]:
Opiates = ['Codeine', 'Dihydrocodeine', 'Tramadol', 'Tapentadol', 'Buprenorphine', 'Fentanyl', 'Methadone', 'Morph', 'Oxycodone']

# commented because have run to Feb 2014
# for opiate in Opiates:
#   pattern_timeseries(opiate)
  

In [ ]:
#load prev outputs from above
pathtopickles = '/media/drcjar/pidisk/prescribing_data/pickles/'
os.chdir(pathtopickles)

Codeine = pd.read_pickle('Codeine.pkl')
Dihydrocodeine = pd.read_pickle('Dihydrocodeine.pkl')
Tramadol = pd.read_pickle('Tramadol.pkl')
Tapentadol = pd.read_pickle('Tapentadol.pkl')
Buprenorphine = pd.read_pickle('Buprenorphine.pkl')
Fentanyl = pd.read_pickle('Fentanyl.pkl')
Methadone = pd.read_pickle('Methadone.pkl')
Morph = pd.read_pickle('Morph.pkl')
Oxycodone = pd.read_pickle('Oxycodone.pkl')
Fesoterodine = pd.read_pickle('Fesoterodine.pkl')

In [ ]:
Codeine.head()

In [ ]:
# work out quantity in mg from prep information and quantity
Codeine['BNF_NAME'] = Codeine['BNF_NAME'].map(str.strip) # get rid of white space
codeine_preps = Codeine['BNF_NAME'].unique().tolist() # get a list of preparation names
codeine_doses = [3, 3, 0.6, 60, 5, 15, 30, 60, 3.2, 12.8, 8, 1.35, 12.8, 2, 8, 6, 1, 0.6, 10, 5, 30, 15, 30, 12] #codeine in mg for each prep
codeine_dose_lookup = dict(zip(codeine_preps, codeine_doses)) # make dict of prep names and codeine doses
Codeine['QUANTITY_IN_MG'] = Codeine['BNF_NAME'].map(lambda x: codeine_dose_lookup[x]) * Codeine['QUANTITY']

In [ ]:
Codeine.PRACTICE.nunique() # number of unique practices in the prescribing dataset

In [ ]:
# get details on practices including practice size and which ccg the practice belongs to into a handy format

pathtogpdata = '/home/drcjar/Documents/mapping/'
os.chdir(pathtogpdata)

# prepare GP pop data
gppop = pd.read_csv('GP_registered_patients_2012.csv')
gppop['Population'] = gppop['Population'].str.replace(',', '') # clean file
rows_with_stars = gppop['Population'].str.contains('\*') # get the rows with * for population
gppop = gppop[~rows_with_stars] # throw away rows with * for population
gppop['Population'] = gppop['Population'].astype(int) # make the occasional strings in this numerical field into ints
gppop = gppop.groupby('GP_Code')
gppop = gppop.Population.sum() # because have age strata pops but practice level prescribing
gppop = DataFrame(gppop).reset_index()
gppop.columns = ['PRACTICE', 'Population']

# add gp names
gpname = pd.read_csv('PracticeProfiles.csv', usecols=[0,1])

# prepare practice to ccg code data
gp2ccg = pd.read_csv('practice_to_ccg_codes.csv')
gpname.columns = ['PRACTICE', 'Practice Name']
gppop = pd.merge(gppop, gpname, on='PRACTICE')

# merge GP pop and practice to ccg code data
gp2ccg = pd.merge(gppop, gp2ccg, on='PRACTICE')

In [ ]:
gppop

In [ ]:
CodeineCCG = pd.merge(Codeine, gp2ccg, on='PRACTICE') # merge our prescribing data with our gp infos

In [ ]:
CodeineCCG.head()

In [ ]:
CodeineCCG.PRACTICE.nunique() # we do not have pop/name/ccg data for all practices 9688 -> 7946

In [ ]:
CodeineCCG['QUANTITY_IN_MG_PER_PERSON'] = CodeineCCG['QUANTITY_IN_MG'] / CodeineCCG['Population']
CodeineCCG['QUANTITY_IN_MG_PER_PERSON'].max() # what's the most mg of codeine prescribed per person per month?

In [ ]:
CodeineCCG['QUANTITY_IN_MG_PER_PERSON'].describe()

In [ ]:
CodeineCCG[CodeineCCG['QUANTITY_IN_MG_PER_PERSON'] > 200]  # who's that?

In [ ]:
CodeineCCG.index = CodeineCCG.DateTime

In [ ]:
PracticeMeanCodeinePerPerson = CodeineCCG.groupby('PRACTICE')['QUANTITY_IN_MG_PER_PERSON'].mean()

In [ ]:
PracticeMeanCodeinePerPerson.sort()
PracticeMeanCodeinePerPerson.head()

In [ ]:
PracticeMeanCodeinePerPerson.tail()

In [ ]:
PracticeMeanCodeinePerPerson.median()

In [ ]:
PracticeMeanCodeinePerPerson.describe()

In [ ]:
plt.title('Codeine Prescribed Per Person by GP Practice for England August 2010 - February 2014 (N=8044)')
plt.ylabel('Number of practices')
plt.xlabel('Mean Codeine Prescribed Per Person (mg)')

PracticeMeanCodeinePerPerson.hist(bins=8044, figsize=(8,8))

In [ ]:
plt.title('Monthly Codeine Prescribed Per Person for GP Practices in England \n August 2010 - February 2014 (N=8044)')
plt.ylabel('Mean Codeine Prescribed Per Person (mg)')

plt.ylim((0,5))
CodeineCCG.groupby('DateTime').QUANTITY_IN_MG_PER_PERSON.mean().plot(linewidth=2.0, figsize=(8,8))


In [ ]:
CodeineCCGts = CodeineCCG.groupby('DateTime').QUANTITY_IN_MG_PER_PERSON.mean()
CodeineCCGts = DataFrame(CodeineCCGts).reset_index()

plt.ylim((0,5))
CodeineCCGts.boxplot()

In [ ]:
# get some ccg information datas involved

pathtoccgdata = '/home/sam/Documents/OpenDataAbstract/'
os.chdir(pathtoccgdata)
ccg_pop = pd.read_csv('ccgcode_pop.csv')
ccg_names = pd.read_csv('CCG_name.csv')

In [ ]:
ccg_names = ccg_names.drop_duplicates() # deduplicate for merging

In [ ]:
ccg_pop = pd.merge(ccg_pop, ccg_names, on='CCGCODE') # merge the info datas
ccg_pop.head()


In [ ]:
grouped = CodeineCCG.groupby(['CCG13CD', 'DateTime']).QUANTITY_IN_MG.sum() # group by ccg and date time, sum the mg of codeine
grouped = DataFrame(grouped).reset_index()
grouped2 = pd.merge(grouped, ccg_pop, on='CCG13CD')

    

In [ ]:
grouped2['QUANTITY_IN_MG_PER_PERSON'] = grouped2['QUANTITY_IN_MG'] / grouped2['Population'] # make it per person using ccg pop data
grouped2.index = grouped2.DateTime

grouped3 = grouped2['2014-02-01'] # lets just have a look at most recent data
# grouped3.QUANTITY_IN_MG_PER_PERSON.sort()

grouped3.sort(columns='QUANTITY_IN_MG_PER_PERSON').describe() # sort and do summary stats


In [ ]:
grouped3.sort(columns='QUANTITY_IN_MG_PER_PERSON').min() # which ccg has lowest per person mg of codeine prescribe?

In [ ]:
grouped3.sort(columns='QUANTITY_IN_MG_PER_PERSON').max() # which ccg has highest per person mg of codeine prescribe?

In [ ]:
grouped3.sort(columns='QUANTITY_IN_MG_PER_PERSON').median()

In [ ]:
grouped3['CCG_Name'].count() #210 CCGs

In [ ]:
plt.title('Codeine prescribed per person by CCG (N=210)')
plt.ylabel('Number of practices')
plt.xlabel('(mg)')

grouped3['QUANTITY_IN_MG_PER_PERSON'].hist(bins=210)

In [ ]:
plt.title('Codeine prescribed per person by CCG (N=210)')
plt.ylabel('(mg)')


grouped3.boxplot(column='QUANTITY_IN_MG_PER_PERSON')

In [ ]:
ccg_geo = 'ccgs.json'

map = folium.Map(location=[54.2, -2.45], zoom_start=5)
map.geo_json(geo_path=ccg_geo, data_out='Feb2014CodeineByCCG.json', data=grouped3,
      columns=['CCG13CD', 'QUANTITY_IN_MG_PER_PERSON'],
      key_on='feature.properties.CCG13CD',
      fill_color='PuBu', fill_opacity=0.7, line_opacity=0.3,
      legend_name='Codeine prescribed per person by CCG for Feb 2014 (mg)')
map.create_map(path='Feb2014CodeineByCCG.html')

In [ ]:
# from IPython.display import IFrame
# IFrame('http://127.0.0.1:8000/Feb2014CodeineByCCG.html', width=700, height=350)
# blocks.org is fiddly with a big ccg.json..

In [ ]:
from IPython.display import Image
Embed = Image('opiateanalysis.png')
Embed


In [ ]:
for opiate in Opiates:
  print opiate

In [ ]:
for opiate in Opiates:
    df = pd.DataFrame(eval(opiate).BNF_NAME.unique())
    df['mg of drug per unit of preparation'] = 'tbc'
    df.to_csv('%s_for_Luke_to_complete.csv' % opiate)

In [ ]:
import xlrd 
import csv

def csv_from_excel(excel_file):
    
    pathtoxls = "/media/mydisk/prescribing_data/xls"
    pathtocsv = "/media/mydisk/prescribing_data/xls/csv"
    os.chdir(pathtoxls)

    workbook = xlrd.open_workbook(excel_file)
    all_worksheets = workbook.sheet_names()
    for worksheet_name in all_worksheets:
        worksheet = workbook.sheet_by_name(worksheet_name)
        your_csv_file = open(''.join([worksheet_name,'.csv']), 'wb')
        wr = csv.writer(your_csv_file, quoting=csv.QUOTE_ALL)

        for rownum in xrange(worksheet.nrows):
            wr.writerow([unicode(entry).encode("utf-8") for entry in worksheet.row_values(rownum)])
        your_csv_file.close()
        
   

In [ ]:
csv_from_excel('/media/mydisk/prescribing_data/xls/Opiates.xls')


In [ ]:
cd /media/mydisk/prescribing_data/xls/csv

In [ ]:
#Buprenorphine
#work out quantity in mg from prep information and quantity
df = pd.read_csv('Buprenorphine.csv') #lookup prepared by colleague
Buprenorphine['BNF_NAME'] = Buprenorphine['BNF_NAME'].map(str.strip) #get rid of white space
buprenorphine_preps = Buprenorphine['BNF_NAME'].unique().tolist() #get a list of preparation names
buprenorphine_doses = df['mg of drug per unit of preparation'].str.replace('mg', '').astype(float).tolist() #remove the mg my collaborator added
buprenorphine_dose_lookup = dict(zip(buprenorphine_preps, buprenorphine_doses)) #make dict of prep names and codeine doses
Buprenorphine['QUANTITY_IN_MG'] = Buprenorphine['BNF_NAME'].map(lambda x: buprenorphine_dose_lookup[x]) * Buprenorphine['QUANTITY']

In [ ]:
#Dihydrocodeine
#work out quantity in mg from prep information and quantity
df = pd.read_csv('Dihydrocodeine.csv') #lookup prepared by colleague
Dihydrocodeine['BNF_NAME'] = Dihydrocodeine['BNF_NAME'].map(str.strip) #get rid of white space
dihydrocodeine_preps = Dihydrocodeine['BNF_NAME'].unique().tolist() #get a list of preparation names
dihydrocodeine_doses = df['mg of drug per unit of preparation'].str.replace('mg', '').astype(float).tolist() #remove the mg my collaborator added
dihydrocodeine_dose_lookup = dict(zip(dihydrocodeine_preps, dihydrocodeine_doses)) #make dict of prep names and codeine doses
Dihydrocodeine['QUANTITY_IN_MG'] = Dihydrocodeine['BNF_NAME'].map(lambda x: dihydrocodeine_dose_lookup[x]) * Dihydrocodeine['QUANTITY']

In [ ]:
#Tramadol
#work out quantity in mg from prep information and quantity
df = pd.read_csv('Tramadol.csv') #lookup prepared by colleague
Tramadol['BNF_NAME'] = Tramadol['BNF_NAME'].map(str.strip) #get rid of white space
tramadol_preps = Tramadol['BNF_NAME'].unique().tolist() #get a list of preparation names
tramadol_doses = df['mg of drug per unit of preparation'].tolist() #remove the mg my collaborator added
tramadol_dose_lookup = dict(zip(tramadol_preps, tramadol_doses)) #make dict of prep names and codeine doses
Tramadol['QUANTITY_IN_MG'] = Tramadol['BNF_NAME'].map(lambda x: tramadol_dose_lookup[x]) * Tramadol['QUANTITY']

In [ ]:
#Tapentadol
#work out quantity in mg from prep information and quantity
df = pd.read_csv('Tapentadol.csv') #lookup prepared by colleague
Tapentadol['BNF_NAME'] = Tapentadol['BNF_NAME'].map(str.strip) #get rid of white space
tapentadol_preps = Tapentadol['BNF_NAME'].unique().tolist() #get a list of preparation names
tapentadol_doses = df['mg of drug per unit of preparation'].tolist() #remove the mg my collaborator added
tapentadol_dose_lookup = dict(zip(tapentadol_preps, tapentadol_doses)) #make dict of prep names and codeine doses
Tapentadol['QUANTITY_IN_MG'] = Tapentadol['BNF_NAME'].map(lambda x: tapentadol_dose_lookup[x]) * Tapentadol['QUANTITY']

In [ ]:
#Fentanyl
#work out quantity in mg from prep information and quantity
df = pd.read_csv('Fentanyl.csv') #lookup prepared by colleague
Fentanyl['BNF_NAME'] = Fentanyl['BNF_NAME'].map(str.strip) #get rid of white space
fentanyl_preps = Fentanyl['BNF_NAME'].unique().tolist() #get a list of preparation names
fentanyl_doses = df['mg of drug per unit of preparation'].tolist() #remove the mg my collaborator added
fentanyl_dose_lookup = dict(zip(fentanyl_preps, fentanyl_doses)) #make dict of prep names and codeine doses
Fentanyl['QUANTITY_IN_MG'] = Fentanyl['BNF_NAME'].map(lambda x: fentanyl_dose_lookup[x]) * Fentanyl['QUANTITY']

In [ ]:
#Methadone
#work out quantity in mg from prep information and quantity
df = pd.read_csv('Methadone.csv') #lookup prepared by colleague
Methadone['BNF_NAME'] = Methadone['BNF_NAME'].map(str.strip) #get rid of white space
methadone_preps = Methadone['BNF_NAME'].unique().tolist() #get a list of preparation names
methadone_doses = df['mg of drug per unit of preparation'].str.replace('tbc', '0')#remove the mg my collaborator added
methadone_doses = methadone_doses.astype(float).tolist()
methadone_dose_lookup = dict(zip(methadone_preps, methadone_doses)) #make dict of prep names and codeine doses
Methadone['QUANTITY_IN_MG'] = Methadone['BNF_NAME'].map(lambda x: methadone_dose_lookup[x]) * Methadone['QUANTITY']

In [ ]:
#Morphine
#work out quantity in mg from prep information and quantity
df = pd.read_csv('Morphine.csv') #lookup prepared by colleague
Morph['BNF_NAME'] = Morph['BNF_NAME'].map(str.strip) #get rid of white space
morphine_preps = Morph['BNF_NAME'].unique().tolist() #get a list of preparation names
morphine_doses = df['mg of drug per unit of preparation'].str.replace('tbc', '0')
morphine_doses = morphine_doses.astype(float).tolist() #remove the mg my collaborator added
morphine_dose_lookup = dict(zip(morphine_preps, morphine_doses)) #make dict of prep names and codeine doses
Morph['QUANTITY_IN_MG'] = Morph['BNF_NAME'].map(lambda x: morphine_dose_lookup[x]) * Morph['QUANTITY']

In [ ]:
#Oxycodone
#work out quantity in mg from prep information and quantity
df = pd.read_csv('Oxycodone.csv') #lookup prepared by colleague
Oxycodone['BNF_NAME'] = Oxycodone['BNF_NAME'].map(str.strip) #get rid of white space
oxycodone_preps = Oxycodone['BNF_NAME'].unique().tolist() #get a list of preparation names
oxycodone_doses = df['mg of drug per unit of preparation'].tolist()
oxycodone_dose_lookup = dict(zip(oxycodone_preps, oxycodone_doses)) #make dict of prep names and codeine doses
Oxycodone['QUANTITY_IN_MG'] = Oxycodone['BNF_NAME'].map(lambda x: oxycodone_dose_lookup[x]) * Oxycodone['QUANTITY']

In [ ]:
Opiates

In [ ]:
#do what we should have done ages ago and concat the dataframes
df_list = [Codeine, Dihydrocodeine, Tramadol, Tapentadol, Buprenorphine, Fentanyl, Methadone, Morph, Oxycodone]
AllOpiates = pd.concat(df_list) 

#add gp to ccg datas
AllOpiatesCCG = pd.merge(AllOpiates, gp2ccg, on='PRACTICE')

#add per person quantity using gp pops
AllOpiatesCCG['QUANTITY_IN_MG_PER_PERSON'] = AllOpiatesCCG['QUANTITY_IN_MG'] / AllOpiatesCCG['Population']

#save the output
os.chdir(pathtopickles)
AllOpiates.to_pickle('AllOpiates.pkl')
AllOpiatesCCG.to_pickle('AllOpiatesCCG.pkl')


In [ ]:
#load pickles
pathtopickles = '/media/mydisk/prescribing_data/pickles/'
os.chdir(pathtopickles)

AllOpiates = pd.read_pickle('AllOpiates.pkl')
AllOpiatesCCG = pd.read_pickle('AllOpiatesCCG.pkl')

In [ ]:
Allgrpd = AllOpiatesCCG.groupby(['CCG13CD', 'DateTime']).QUANTITY_IN_MG.sum() #group by ccg and date time, sum the mg of codeine
Allgrpd = DataFrame(Allgrpd).reset_index()
Allgrpd2 = pd.merge(Allgrpd, ccg_pop, on='CCG13CD')
Allgrpd2['QUANTITY_IN_MG_PER_PERSON'] = Allgrpd2['QUANTITY_IN_MG'] / Allgrpd2['Population'] 


In [ ]:
Allgrpd2.QUANTITY_IN_MG_PER_PERSON.describe()

In [ ]:
Allgrpd2.index = Allgrpd2.DateTime
plt.title('Monthly mg per person of all opiates presribed by CCG')
plt.ylim((0,400))


for ccg in Allgrpd2.CCG_Name.unique()[:5]:
    Allgrpd2[Allgrpd2['CCG_Name'] == ccg].QUANTITY_IN_MG_PER_PERSON.plot(label=ccg, figsize=(9,9))
    
legend(loc='upper left', frameon=False )


In [ ]:
Allgrpd3 = Allgrpd2['2014-02-01'] #cut of most recent data

In [ ]:
Allgrpd3.QUANTITY_IN_MG_PER_PERSON.describe()

In [ ]:
Allgrpd3.QUANTITY_IN_MG_PER_PERSON.median()

In [ ]:
Allgrpd3.head()

In [ ]:
Allgrpd3.boxplot(column='QUANTITY_IN_MG_PER_PERSON')

In [ ]:
Allgrpd3.QUANTITY_IN_MG_PER_PERSON.hist()

In [ ]:
import folium
ccg_geo = 'ccgs.json'

map = folium.Map(location=[54.2, -2.45], zoom_start=5)
map.geo_json(geo_path=ccg_geo, data_out='OpiateByCCG.json', data=Allgrpd3,
      columns=['CCG13CD', 'QUANTITY_IN_MG_PER_PERSON'],
      key_on='feature.properties.CCG13CD',
      fill_color='PuBu', fill_opacity=0.7, line_opacity=0.3,
      legend_name='Opiates prescribed per person by CCG for Feb 2014 (mg)')
map.create_map(path='Feb2014OpiatesByCCG.html')

In [ ]:
from IPython.display import Image
Embed = Image('opiateanalysis2.png')
Embed


In [ ]:
AllOpiatesCCG.groupby('PRACTICE').QUANTITY_IN_MG_PER_PERSON.max().max()

In [ ]:
AllOpiatesCCG[AllOpiatesCCG['QUANTITY_IN_MG_PER_PERSON'] > 300][:5] #who's that?

In [ ]:
AllOpiatesCCG.PRACTICE.nunique()

In [ ]:
Allgrpd2.groupby(['DateTime', 'CCG_Name'])['QUANTITY_IN_MG_PER_PERSON'].sum().to_csv('ccgopiatetotals.csv')

In [ ]:
Allgrpd2.groupby(['DateTime', 'CCG_Name'])['QUANTITY_IN_MG_PER_PERSON'].sum()

In [ ]:
for i, item in enumerate(df_list):
    df_list[i] = pd.merge(df_list[i], gp2ccg, on='PRACTICE')
    df_list[i] = df_list[i].groupby(['CCG13CD', 'DateTime']).QUANTITY_IN_MG.sum() #group by ccg and date time, sum the mg of codeine
    df_list[i] = DataFrame(df_list[i]).reset_index()
    df_list[i] = pd.merge(df_list[i], ccg_pop, on='CCG13CD')
    df_list[i]['QUANTITY_IN_MG_PER_PERSON'] = df_list[i]['QUANTITY_IN_MG'] / df_list[i]['Population'] 
    df_list[i].QUANTITY_IN_MG_PER_PERSON.describe()

In [ ]:
for i, item in enumerate(df_list):
    df_list[i].groupby(['DateTime', 'CCG_Name'])['QUANTITY_IN_MG_PER_PERSON'].sum().to_csv('%stotals.csv' % Opiates[i])

In [ ]:
ls GP*

In [ ]:
gps = pd.read_csv('GP_registered_patients_2012.csv')


In [ ]:
gps.CCG_Code.nunique()

In [ ]:
df